In [8]:
import numpy as np
import pandas as pd
import glob
import random
import os

import open3d as o3d
from pyntcloud import PyntCloud

In [1]:
from mine_clustering import Kitti_utils
import imp
imp.reload(Kitti_utils)

<module 'mine_clustering.Kitti_utils' from 'D:\\xr\\study\\point cloud\\FinalProject\\mine_clustering\\Kitti_utils.py'>

## 1. Prepare

* I assign all the threshold(confidence) as 0.8, as I forget to add it in the result file.
* as I set:  x1=x2 y1=y2, I need to change line 459 to (otherwise all the result will be filtered by this):

    if(false)//height<MIN_HEIGHT[difficulty]
   
* Run eval result: 
./evaluate_object_3d_offline /home/viki/Documents/DeepBlue/PointCloud/result_test/label_2 /home/viki/Documents/DeepBlue/PointCloud/result_test/mine_result/


## 2. The evaluation result

### bird view

Car  | Cyclist | Pedestrian 
:-:|:- |-:  
![image](mine_result/plot/Car_detection_ground.png)|![image](mine_result/plot/Cyclist_detection_ground.png)|![image](mine_result/plot/Pedestrian_detection_ground.png)

### 3d bounding box 

Car  | Cyclist | Pedestrian 
:-:|:- |-:  
![image](mine_result/plot/Car_detection_3d.png)|![image](mine_result/plot/Cyclist_detection_3d.png)|![image](mine_result/plot/Pedestrian_detection_3d.png)

#### The results are far from ideal, (far from our classification test result : with the precision 90%). And the analysis of the results follows.

## 3. Result analysis

To investigate the result, we show some of our detection.

In [6]:
folder_ground_truth = 'D:/xr/study/point cloud/Dataset/training/label_2'
folder_point_cloud = 'D:/xr/study/point cloud/Dataset/training/velodyne'
mine_result_dir = './mine_result/data'
files_mine_results = glob.glob(mine_result_dir + '/*.txt')
print("There are ", len(files_mine_results), " files")

There are  7481  files


In [10]:
# test
idx = 500
print(files_mine_results[idx])
show_sets, file_velodyne = Kitti_utils.load_ground_true(folder_point_cloud, files_mine_results[idx])

# test show
point_cloud_test = Kitti_utils.load_velo_scan(file_velodyne)
print(" point cloud shape is : ", point_cloud_test.shape)
point_cloud_o3d = o3d.geometry.PointCloud()
point_cloud_o3d.points = o3d.utility.Vector3dVector(point_cloud_test[:,0:3])
show_sets.append(point_cloud_o3d)
o3d.visualization.draw_geometries(show_sets)

./mine_result/data\000500.txt
 point cloud shape is :  (113162, 4)


I have checked lots of my result, can I find (with example shown):
#### a. Some results are fine, especially the cars, are correctly found. For an example in the following image :

![image](images/fine_result.png)

#### b. However, there exist a lot of noise detections, for example, in the image below :

![image](images/noise_result.png)

These noise are most the objects should be classified as "Others". And they are the main contribution to the false detections. This is mainly introduced by the data set making process :

* We a lot more "Other" data than normal data. Should be the result of our definition of problem, and also a result of a not-good-enough Segmentation algorithm.
* In the data set making process, I have picked more large clouds than small clouds, which lead to a bad classification of the small clouds. And it is hard, as each seperated cloud has much fewer feature than in the whole map (To be honest, it is hard to classify them even by experts).

#### Todo. We should better build a more elaborated data set for training. 

* Build a better segmentation algorithm.
* We could include more classes for better classification. For an example, class "Other - plants", "Other - building", "Other - Other", etc.
* We should use a more sophisticated data augmentation, and a better method to equalize the data.
* We could introduce a feature engineering process to include more features, for example its height with respect to the ground, its distance to the closest detected road, etc.
* Could we find a way to include the infomation of the relationship of the cloud segments ?
